In [1]:
# TODOS:
# Add warnings if expected directory or files do not exist
# Comment heavily

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import ecephys_analyses as ea

yaml_path = ea.package_datapath('example_org_spec.yaml')

In [5]:
import yaml

with open(yaml_path) as fp:
    yaml_data = yaml.safe_load(fp)
doc = yaml_data

In [110]:
from pathlib import Path

session = doc['recording-sessions'][0]
run = '3-1-2021_A'
gate = 'g1'
probe = 'imec0'
fname = '3-1-2021_A_g1_t0.imec0.lf.meta'
root_dir = Path(doc['raw-data-root'])
session_dir = root_dir / session['directory']
gate_dir = session_dir / f"{run}_{gate}"
probe_dir = Path(doc['raw-data-root'], session['directory'], f"{run}_{gate}", f"{run}_{gate}_{probe}")

# Test ecephys.sglx.paths

In [109]:
import ecephys.sglx.paths as sglx_paths

In [19]:
sglx_paths.parse_sglx_fname(fname)

('3-1-2021_A', 'g1', 't0', 'imec0', 'lf', 'meta')

In [20]:
sglx_paths.get_trigger_files(probe_dir)

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.lf.bin'),
 Po

In [21]:
sglx_paths.get_unique_trigger_stems(probe_dir)

['3-1-2021_A_g1_t0', '3-1-2021_A_g1_t1']

In [22]:
sglx_paths.get_probe_directories(gate_dir)

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec1')]

In [23]:
sglx_paths.get_probe_directories(gate_dir, probe_regex="imec0")

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0')]

In [24]:
sglx_paths.get_gate_files(gate_dir)

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.meta'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.lf.bin'),
 Po

In [25]:
sglx_paths.filter_files(sglx_paths.get_gate_files(gate_dir), probe='imec0', stream='lf', ftype='bin')

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.bin'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.lf.bin')]

In [26]:
sglx_paths.remove_suffixes(sglx_paths.get_gate_files(gate_dir))

[PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec1/3-1-2021_A_g1_t0'),
 PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec1/3-1-2021_A_g1_t1')]

In [27]:
sglx_paths.separate_files_by_probe(sglx_paths.get_gate_files(gate_dir))

{'imec0': [PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.bin'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.ap.meta'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.bin'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t0.imec0.lf.meta'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.bin'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.imec0.ap.meta'),
  PosixPath('/Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g1/3-1-2021_A_g1_imec0/3-1-2021_A_g1_t1.ime

# Test ecephys.sglx.file_selection

In [68]:
import ecephys.sglx.file_selection as mgmt

In [87]:
mgmt._get_gate_directories(session_dir, run);

In [82]:
mgmt._get_run_files(session_dir, run);

In [83]:
mgmt._get_session_files(root_dir, session);

In [86]:
mgmt._get_document_files(doc);

In [93]:
mgmt.get_run_files(session_dir, run);

In [95]:
mgmt.get_session_files(root_dir, session);

In [97]:
mgmt.get_document_files(doc);

In [101]:
mgmt._get_experiment_files(doc, 'sleep-homeostasis');

In [104]:
mgmt.get_experiment_files(doc, 'sleep-homeostasis');

In [107]:
mgmt.get_alias_files(doc, 'sleep-homeostasis', 'light-period-circadian-match');

# Under development

In [ ]:
#SGLX schema + agnostic files should be in their own file, probably in sglx_utils
#Functions that extend SGLX schema should be in a separate file, in ecephys_analyses, and should return dataframes